In [50]:
from datetime import datetime
from typing import List, Optional
from pydantic import BaseModel

class User(BaseModel):
    id: int
    name = 'John Doe'
    signup_ts: Optional[datetime] = None
    friends: List[int] = []

external_data = {
    'id': '123',
    'signup_ts': '2017-06-01 12:22',
    'friends': [1, '2', b'3'],
}

user = User(**external_data)
print(user.id)
#> 123
print(repr(user.signup_ts))
#> datetime.datetime(2017, 6, 1, 12, 22)
print(user.friends)
#> [1, 2, 3]
print(user.dict())


123
datetime.datetime(2017, 6, 1, 12, 22)
[1, 2, 3]
{'id': 123, 'name': 'John Doe', 'signup_ts': datetime.datetime(2017, 6, 1, 12, 22), 'friends': [1, 2, 3]}


In [2]:
user

<User id=123 name='John Doe' signup_ts=datetime.datetime(2017, 6, 1, 12, 22) friends=[1, 2, 3]>

In [48]:
from pydantic import ValidationError

try:
    User(signup_ts='broken', friends=[1, 2, 'not number'])
except ValidationError as e:
    print(e)

4 validation errors
id
  field required (type=value_error.missing)
signup_ts
  invalid datetime format (type=type_error.datetime)
signup_ts
  value is not none (type=type_error.none.allowed)
friends -> 2
  value is not a valid integer (type=type_error.integer)


Now this is another tutorial from [here](https://techtutorialsx.com/2022/06/03/pydantic-getting-started/)

In [7]:
import pydantic
from pydantic import BaseModel, ValidationError

# Now defining the Person class
class Person(BaseModel):
    name: pydantic.StrictStr 
    age: pydantic.StrictInt 
    is_married: pydantic.StrictBool

In [8]:
# Now to create a dictionary that contains the data we want to parse and validate
data ={
    'name': 32,
    'age': True,
    'is_married': 'hhh'
}

In [9]:
try:
    # Now to parse the data and validate it
    person = Person(**data)
except ValidationError as e:
    print(e)

3 validation errors for Person
name
  str type expected (type=type_error.str)
age
  value is not a valid integer (type=type_error.integer)
is_married
  value is not a valid boolean (type=value_error.strictbool)


Another example. Using error class in pydantic to better handle the situations

In [18]:
from pydantic import BaseModel, ValidationError

class Person(BaseModel):
    name: pydantic.StrictStr
    age: pydantic.StrictInt
    is_married: pydantic.StrictBool

data = {
    'age': 'test',
    'is_married': False
}

try:
    person = Person(**data)
    print(person.dict())

except ValidationError as e:
    errors = e.errors()
    print(errors)

    print()
    print(errors[0])

    print()
    print(errors[1]['loc'])
    print(errors[1]['msg'])
    print(errors[1]['type'])

[{'loc': ('name',), 'msg': 'field required', 'type': 'value_error.missing'}, {'loc': ('age',), 'msg': 'value is not a valid integer', 'type': 'type_error.integer'}]

{'loc': ('name',), 'msg': 'field required', 'type': 'value_error.missing'}

('age',)
value is not a valid integer
type_error.integer


In [13]:
print(person)

NameError: name 'person' is not defined

## Adding Lists to the Pydantic module

In [19]:
from typing import List

class Address(BaseModel):
    street: pydantic.StrictStr
    Building: pydantic.StrictInt

# Now enhancing the Person class
class Person(BaseModel):
    name: pydantic.StrictStr
    age: pydantic.StrictInt
    is_married: pydantic.StrictBool
    address: Address
    languages: List[pydantic.StrictStr]

In [20]:
data = {
    'age':10,
    'name':'John',
    'is_married':False,
    'address':{
        'street':'Main Street',
        'Building': 10
    },
    'languages': ['pt-pt', 'en-us']
}


In [23]:
try:
    person = Person(**data)
    print(person.dict())

except ValidationError as e:
    print('Exception as str: ')
    print(e)
    print('Exception as json: ')
    print(e.json())

{'name': 'John', 'age': 10, 'is_married': False, 'address': {'street': 'Main Street', 'Building': 10}, 'languages': ['pt-pt', 'en-us']}


In [24]:
# Checking some errors in data
data = {
    'age': 10,
    'name': 'John',
    'is_married': False,
    'address': {
        'street': 'st street',
        'building': 'test'
    },
    'languages':[{}, 'en-us']
}

In [26]:
try:
    person = Person(**data)
    print(person.dict())

except ValidationError as e:
    print('Exception as str: ')
    print(e)
    print('Exception as json: ')
    print(e.json())

Exception as str: 
2 validation errors for Person
address -> Building
  field required (type=value_error.missing)
languages -> 0
  str type expected (type=type_error.str)
Exception as json: 
[
  {
    "loc": [
      "address",
      "Building"
    ],
    "msg": "field required",
    "type": "value_error.missing"
  },
  {
    "loc": [
      "languages",
      0
    ],
    "msg": "str type expected",
    "type": "type_error.str"
  }
]
